In [ ]:
# You need to run this cell to get things setup
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as pyplot

# Introducing geopandas
During this course we will often use the [geopandas](http://geopandas.org/) package to explore handling spatial data programatically. 

That name probably requires at least a little bit of explanation. [Pandas](http://pandas.pydata.org/index.html) is an advanced package for manipulating tables of data.  The name is an approximate acronym for Python Data Analysis library.  Anyway, `geopandas` combines pandas (data tables) with number of other packages, in particular, shapely (https://pypi.python.org/pypi/Shapely) which deals with geometry and fiona (https://pypi.python.org/pypi/Fiona) which handles spatial data formats.

So... let's import geopandas.

In [ ]:
import geopandas

Like `import math` this gives us access to a specialised collection of functions and objects, this time for dealing with spatial data.

## Opening a spatial data file
This is simple. The `read_file()` function will do the trick, and provided the file format makes sense, and is consistent with the file extension, you should have no issues.

In [ ]:
counties = geopandas.read_file('ca-counties.shp')

Assign the file read result to a suitable variable name, and from there on you are dealing with a geopandas `GeoDataFrame` object, which allows us to examine the contents.  The `.head()` function will show us the first few rows of the data table.

In [ ]:
counties.head()

The important thing to notice here, is the **geometry** column in the table. It may not show up in the table view above unless you scroll to the right. Let's take a look at it

In [ ]:
counties.geometry.head()

We can also use an alternative notation to access individual columns in the data table.

In [ ]:
counties['geometry'].head()

What is a geometry? 

In [ ]:
counties.geometry[0]

Cool! You just made a map of Lassen County, California. Not a particularly useful map, but a map nevertheless.

The geometry column in a `GeoDataFrame` can store points, polygons, line segments or a variety of other `shapely` geometries, which can then be manipulated and managed in various ways according to our needs.

### Let's make some maps!

This turns out to be very simple, at least for a basic map.

In [ ]:
counties.plot()

Kinda small. Try again, this time using `figsize` to specify a width and height. The units are inches, and the final size of plot will depend on the screen resolution, so this is not a simple matter. For now, just know that you can change the map size.

In [ ]:
counties.plot(figsize=(6,9))

We can manipulate the look of this with a variety of other options in the `plot()` function. For example...

In [ ]:
counties.plot(figsize=(6,9), linewidth=0.25, edgecolor='w', facecolor='lightgray')

You can find out more about the plotting options for basic maps in geopandas from [the documentation](http://geopandas.org/mapping.html) and also from the [matplotlib documentation](http://matplotlib.org/api/pyplot_api.html). 

For closer control over things, you need to a work at a lower level with the [`pyplot` API](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot). For example, to put county names on the map, this will work.

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(111)
for polygon, name in zip(counties.geometry, counties.NAME):
    ax.annotate(xy=(polygon.centroid.x-50000, polygon.centroid.y), s=name)
counties.plot(ax=ax, linewidth=0.25, edgecolor='w', facecolor='lightgray')

Pretty ugly, both the map and the code. There is no getting away from the fact that the plotting libraries available in Python are not much fun to work with (although mostly that is not why we are using Python anyway).

Just to persist with the example, in spite of the difficulties:

The problem is that by working with the `pyplt` low-level API, the plotting is no longer aware that the $(x, y)$ coordinates of the geometries are not just any old $(x, y)$ coordinates, but meaningful ones that need to be treated as such.  In particular, in most cases, they should be treated as scaled equally (in this case they are meters, from an arbitrary origin).  We enforce this by specifying that the plot should employ an equal aspect ratio.

In [ ]:
fig = pyplot.figure(figsize=(6,9))
ax = fig.add_subplot(111)
# Apply equal aspect ratio
ax.set_aspect('equal')
for polygon, name in zip(counties.geometry, counties.NAME):
    ax.annotate(xy=(polygon.centroid.x-50000, polygon.centroid.y), s=name)
counties.plot(ax=ax, linewidth=0.25, edgecolor='w', facecolor='lightgray')

That's still not perfect, but it is a big improvement.

It is possible to make nice maps in geopandas, but to make finished publication quality maps, you will most likely want to export the map to a graphical file format for final tidy up in a package such as Inkscape or Illustrator. We aren't particularly concerned at the moment with publication quality maps so I'll just note the possibility and move on.

In [ ]:
## For example, you could do this, to make an SVG file
fig.savefig('ca.svg')

## Mapping data
Our concern here is with *data* as much as with maps, so how can we map the data in that table we saw above?

We simply tell the `plot()` function what column in the data table to map.  Let's work initially with the `nPop` column which tells us the number of people recorded in the census in each county.

In [ ]:
counties.plot(column='nPop', linewidth=0.25, edgecolor='w', figsize=(6,9))

Not great. One problem here is the color scheme we are using (the default). Try this instead.

In [ ]:
counties.plot(column='nPop', cmap='Reds', linewidth=0.25, edgecolor='k', figsize=(6,9))

Hmmm. What's the problem now? It turns out that in the US, counties are bad units for many kinds of analysis because they have radically different populations. We can see this by making a histogram of the county populations. 

In [ ]:
pyplot.hist(counties.nPop)

There are 53 counties in the state. 49 of them have populations under 1 million (well under that in fact), while Los Angeles county has over 9 million. That very skew distribution means we have to think carefully about how to map such data in a meaningful way. 

Take a look at the code below.

In [ ]:
# set up a figure with three plots 
fig = pyplot.figure(figsize=(15,8))

# 131 means 1 row by 3 columns, 1st plot
ax1 = fig.add_subplot(131)
ax1.set_aspect('equal')
counties.plot(ax=ax1, column='nPop', cmap='Blues', scheme='equal_interval', linewidth=0.25, edgecolor='k', legend=True)

# 131 means 1 row by 3 columns, 2nd plot
ax2 = fig.add_subplot(132)
ax2.set_aspect('equal')
counties.plot(ax=ax2, column='nPop', cmap='Reds', scheme='quantiles', linewidth=0.25, edgecolor='k', legend=True)

# 131 means 1 row by 3 columns, 3rd plot
ax3 = fig.add_subplot(133)
ax3.set_aspect('equal')
counties.plot(ax=ax3, column='nPop', cmap='Greens', scheme='fisher_jenks', linewidth=0.25, edgecolor='k', legend=True)

This makes three maps, each of them using a different *classification scheme* as you are no doubt familiar with.

For now, in the cell below, experiment with making some maps of other columns in the data table.  Experiment with the `edgecolor`, `linewidth` and `facecolor` options.  Also change the color map `cmap` setting.  A list of the available color maps is [provided here](http://matplotlib.org/users/colormaps.html).

## Have an explore

In [ ]:
# Experiment with this code to make other maps
counties.plot(column='nPop', cmap='Reds', linewidth=0.25, edgecolor='k', figsize=(6,9))